# 第3章 字典和集合

字典和集合的实现均依赖于散列表，想要进一步理解集合和字典，就得先理解散列表的原理。

本章的内容大纲如下:

- 常见的字典方法

- 如何处理查找不到的键

- 标准库中dict类型的变种

- set和frozenset类型

- 散列表的工作原理

- 散列表带来的潜在影响（什么样的数据类型可作为键、不可预知的顺序，等等）

## 3.1 泛映射类型

In [ ]:
collections.abc模块中有Mapping和MutableMapping这两个抽象基类，其作用在于为dict和其他类似的类型定义形式接口。

非抽象映射类型一般不会直接继承这些抽象基类，而是直接对dict或collections.User.Dict进行扩展。

In [1]:
# 使用抽象基类和isinstance一起判定某个数据结构是否为广义的映射类型
import collections.abc as abc
mydict = {}
isinstance(mydict, abc.Mapping)

True

标准库里所有的映射类型都是利用dict来实现的，因此他们有个共同的限制，即只有可散列的数据类型才能用作这些映射里的键。

*<b>什么是可散列的数据类型？</b>*

<u>如果一个对象是可散列的，那么在这个对象的生命周期中，它的散列值是不变的，而且这个对象需要实现__hash__()方法。另外可散列的对象还要有__qe__()方法，这样才能跟其他键作比较。</u>

如果两个可散列对象是相等的，那么他们的散列值一定是一样的。

In [2]:
tt = (1, 2, (30, 40))
hash(tt)

8027212646858338501

In [3]:
tl = (1, 2, [30, 40])
hash(tl)

TypeError: unhashable type: 'list'

In [4]:
tf = (1, 2, frozenset([30, 40]))
hash(tf)

985328935373711578

In [7]:
# 字典的构造方法
a = dict(one=1, two=2, three=3)
b = dict({'one':1, "two":2, "three":3})
c = dict(zip(['one', 'two', 'three'], [1, 2, 3]))
d = dict([('one', 1), ('two', 2), ('three', 3)])
e = {'one':1, "two":2, "three":3}

a == b == c == d == e

True

## 3.2 字典推导